In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def get_nba_db():
    """
    Creates a connection to the NBA PostgreSQL database.
    """
    host = os.getenv("NBA_DB_HOST")
    port = os.getenv("NBA_DB_PORT")
    database = os.getenv("NBA_DB_NAME")
    user = os.getenv("NBA_DB_USER")
    password = os.getenv("NBA_DB_PASSWORD")
    
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
    return create_engine(connection_string)

def query(sql):
    """
    Executes SQL query against the NBA database and returns results as a pandas DataFrame.
    """
    engine = get_nba_db()
    return pd.read_sql(sql, engine)

# Function to display available tables
def list_tables():
    """Lists all tables available in the NBA database."""
    tables = query("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
    return tables['table_name'].tolist()

In [2]:
list_tables()

['common_player_info',
 'draft_combine_stats',
 'draft_history',
 'game',
 'game_info',
 'game_summary',
 'games',
 'inactive_players',
 'leagueschedule24_25',
 'leagueschedule25_26',
 'line_score',
 'officials',
 'other_stats',
 'play_by_play',
 'player',
 'players',
 'playerstatistics',
 'team',
 'team_details',
 'team_history',
 'team_info_common',
 'teamhistories',
 'teamstatistics']

In [3]:
query("""
    SELECT * FROM playerstatistics LIMIT 10
""")

,firstname,lastname,personid,gameid,gamedate,playerteamcity,playerteamname,opponentteamcity,opponentteamname,gametype,...,threepointerspercentage,freethrowsmade,freethrowsattempted,freethrowspercentage,reboundsoffensive,reboundsdefensive,reboundstotal,foulspersonal,turnovers,plusminuspoints
0,Chris,Paul,101108.0,22500027.0,2025-10-31T22:30:00Z,LA,Clippers,New Orleans,Pelicans,in-season,...,0.500,0.0,0.0,0.000,1.0,0.0,1.0,0.0,0.0,-1.0
1,Kris,Dunn,1627739.0,22500027.0,2025-10-31T22:30:00Z,LA,Clippers,New Orleans,Pelicans,in-season,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,4.0,1.0,-2.0
2,Ivica,Zubac,1627826.0,22500027.0,2025-10-31T22:30:00Z,LA,Clippers,New Orleans,Pelicans,in-season,...,0.000,4.0,4.0,1.000,1.0,10.0,11.0,5.0,0.0,2.0
3,Derrick,Jones Jr.,1627884.0,22500027.0,2025-10-31T22:30:00Z,LA,Clippers,New Orleans,Pelicans,in-season,...,0.400,0.0,0.0,0.000,2.0,1.0,3.0,1.0,0.0,9.0
4,John,Collins,1628381.0,22500027.0,2025-10-31T22:30:00Z,LA,Clippers,New Orleans,Pelicans,in-season,...,0.333,1.0,1.0,1.000,1.0,2.0,3.0,0.0,1.0,-4.0
5,Zion,Williamson,1629627.0,22500027.0,2025-10-31T22:30:00Z,New Orleans,Pelicans,LA,Clippers,in-season,...,0.000,11.0,13.0,0.846,3.0,3.0,6.0,3.0,3.0,-5.0
6,Jordan,Poole,1629673.0,22500027.0,2025-10-31T22:30:00Z,New Orleans,Pelicans,LA,Clippers,in-season,...,0.538,5.0,5.0,1.000,0.0,2.0,2.0,2.0,5.0,1.0
7,Saddiq,Bey,1630180.0,22500027.0,2025-10-31T22:30:00Z,New Orleans,Pelicans,LA,Clippers,in-season,...,0.000,4.0,4.0,1.000,0.0,4.0,4.0,0.0,0.0,4.0
8,Herbert,Jones,1630529.0,22500027.0,2025-10-31T22:30:00Z,New Orleans,Pelicans,LA,Clippers,in-season,...,0.750,0.0,0.0,0.000,0.0,4.0,4.0,0.0,2.0,-5.0
9,Trey,Murphy III,1630530.0,22500027.0,2025-10-31T22:30:00Z,New Orleans,Pelicans,LA,Clippers,in-season,...,0.500,0.0,0.0,0.000,1.0,4.0,5.0,3.0,1.0,-3.0


In [8]:
sql = """
SELECT 
    personid,
    firstname,
    lastname,
    playerteamname,
    opponentteamname,
    points,
    assists,
    reboundstotal,
    reboundsoffensive,
    reboundsdefensive,
    fieldgoalspercentage,
    threepointerspercentage,
    freethrowspercentage,
    numminutes,
    turnovers,
    foulspersonal,
    plusminuspoints,
    gamedate
FROM playerstatistics
WHERE gamedate >= '2024-01-01'
"""
df = query(sql)
print("Loaded data:", df.shape)

df = df.reset_index()
df['gamedate'] = pd.to_datetime(df['gamedate'], errors='coerce', utc=True).dt.tz_localize(None)
df = df.dropna(subset=['gamedate'])

weekly = (
    df.groupby(['personid', pd.Grouper(key='gamedate', freq='W')])
      .agg({
          'points': 'mean',
          'assists': 'mean',
          'reboundstotal': 'mean',
          'turnovers': 'mean',
          'numminutes': 'mean',
          'plusminuspoints': 'mean',
          'fieldgoalspercentage': 'mean',
          'threepointerspercentage': 'mean',
          'freethrowspercentage': 'mean'
      })
      .reset_index()
)

print("Weekly aggregation complete:", weekly.shape)
display(weekly.head(5))


Loaded data: (64485, 18)
Weekly aggregation complete: (2312, 11)


,personid,gamedate,points,assists,reboundstotal,turnovers,numminutes,plusminuspoints,fieldgoalspercentage,threepointerspercentage,freethrowspercentage
0,2544.0,2025-10-05,0.0,0.0,0.0,0.0,NaN,0.0,0.0000,0.0,0.0
1,2544.0,2025-10-19,0.0,0.0,0.0,0.0,NaN,0.0,0.0000,0.0,0.0
2,42824.0,2025-10-05,0.0,0.0,0.0,0.0,NaN,0.0,0.0000,0.0,0.0
3,101108.0,2025-10-12,8.5,3.0,4.0,1.5,18.885,11.0,0.4335,0.4,0.5
4,101108.0,2025-10-19,8.0,10.0,1.0,1.0,18.260,24.0,0.5000,0.5,0.0


In [ ]:
pd.set_option('display.max_rows', 1000) 
display(weekly)


In [ ]:
# WEEKLY AGGREGATION with MOMENTUM

import pandas as pd

sql = """
SELECT 
    personid,
    firstname,
    lastname,
    playerteamname,
    opponentteamname,
    points,
    assists,
    reboundstotal,
    reboundsoffensive,
    reboundsdefensive,
    fieldgoalspercentage,
    threepointerspercentage,
    freethrowspercentage,
    numminutes,
    turnovers,
    foulspersonal,
    plusminuspoints,
    gamedate
FROM playerstatistics
WHERE gamedate >= '2024-01-01'
"""
df = query(sql)
print(" Loaded data:", df.shape)

df['gamedate'] = pd.to_datetime(df['gamedate'], errors='coerce', utc=True).dt.tz_localize(None)
df = df.dropna(subset=['gamedate'])

weekly = (
    df.groupby(['personid', pd.Grouper(key='gamedate', freq='W')])
      .agg({
          'points': 'mean',
          'assists': 'mean',
          'reboundstotal': 'mean',
          'turnovers': 'mean',
          'numminutes': 'mean',
          'plusminuspoints': 'mean',
          'fieldgoalspercentage': 'mean',
          'threepointerspercentage': 'mean',
          'freethrowspercentage': 'mean'
      })
      .reset_index()
)

print(" Weekly aggregation complete:", weekly.shape)

base_stats = [
    'points', 'assists', 'reboundstotal', 'turnovers', 'numminutes',
    'plusminuspoints', 'fieldgoalspercentage', 'threepointerspercentage', 'freethrowspercentage'
]

for col in base_stats:
    # Rolling 3-week average (momentum)
    weekly[f'{col}_rolling3'] = (
        weekly.groupby('personid')[col].transform(lambda x: x.rolling(3, min_periods=1).mean())
    )

    weekly[f'{col}_diff1'] = (
        weekly.groupby('personid')[col].diff()
    )

    weekly[f'{col}_zscore'] = (
        (weekly[col] - weekly[col].mean()) / weekly[col].std()
    )

weekly['season'] = weekly['gamedate'].dt.year

season_agg = (
    weekly.groupby(['personid', 'season'])[base_stats]
          .agg(['mean', 'sum'])
          .reset_index()
)

season_agg.columns = ['personid', 'season'] + [
    f"{col}_{agg}" for col in base_stats for agg in ['mean', 'sum']
]

weekly = weekly.merge(season_agg, on=['personid', 'season'], how='left')

weekly = weekly.rename(
    columns=lambda x: x.replace('_mean_y', '_season_mean').replace('_sum_y', '_season_sum')
)

print(" Feature engineering complete!")
print(" Final shape:", weekly.shape)
display(weekly.head(10))


 Loaded data: (64485, 18)
 Weekly aggregation complete: (2312, 11)
 Feature engineering complete!
 Final shape: (2312, 57)


,personid,gamedate,points,assists,reboundstotal,turnovers,numminutes,plusminuspoints,fieldgoalspercentage,threepointerspercentage,...,numminutes_mean,numminutes_sum,plusminuspoints_mean,plusminuspoints_sum,fieldgoalspercentage_mean,fieldgoalspercentage_sum,threepointerspercentage_mean,threepointerspercentage_sum,freethrowspercentage_mean,freethrowspercentage_sum
0,2544.0,2025-10-05,0.000000,0.000000,0.000000,0.0,NaN,0.000000,0.000000,0.00,...,NaN,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
1,2544.0,2025-10-19,0.000000,0.000000,0.000000,0.0,NaN,0.000000,0.000000,0.00,...,NaN,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
2,42824.0,2025-10-05,0.000000,0.000000,0.000000,0.0,NaN,0.000000,0.000000,0.00,...,NaN,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
3,101108.0,2025-10-12,8.500000,3.000000,4.000000,1.5,18.885,11.000000,0.433500,0.40,...,15.79625,63.185,6.583333,26.333333,0.322958,1.291833,0.35,1.4,0.208333,0.833333
4,101108.0,2025-10-19,8.000000,10.000000,1.000000,1.0,18.260,24.000000,0.500000,0.50,...,15.79625,63.185,6.583333,26.333333,0.322958,1.291833,0.35,1.4,0.208333,0.833333
5,101108.0,2025-10-26,3.333333,4.666667,1.666667,1.0,14.990,-2.666667,0.233333,0.25,...,15.79625,63.185,6.583333,26.333333,0.322958,1.291833,0.35,1.4,0.208333,0.833333
6,101108.0,2025-11-02,1.500000,3.000000,1.000000,0.5,11.050,-6.000000,0.125000,0.25,...,15.79625,63.185,6.583333,26.333333,0.322958,1.291833,0.35,1.4,0.208333,0.833333
7,200768.0,2025-10-05,0.000000,0.000000,0.000000,0.0,NaN,0.000000,0.000000,0.00,...,NaN,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
8,200768.0,2025-10-26,0.000000,0.000000,0.000000,0.0,NaN,0.000000,0.000000,0.00,...,NaN,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
9,200768.0,2025-11-02,0.000000,0.000000,0.000000,0.0,NaN,0.000000,0.000000,0.00,...,NaN,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
